In [1]:
import re
import os
if not 'id_0123456789876543210' in locals():
    _rootlevel = 2
    _oldwd = re.sub(r'\\', '/', os.getcwd())
    _spdirs = _oldwd.split('/')
    _newwd = '/'.join(_spdirs[:(len(_spdirs)-_rootlevel)])
    os.chdir(_newwd)
    id_0123456789876543210 = None

In [2]:
from src.python.optimize.solution import intBinary, floatBinary, ndSolution, Category, ndCategory
from src.python.optimize.search import LocalSearch, Cooling
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

plt.style.use('ggplot')

# Parametrización de espacio de búsqueda

## Cálculo de valor lambda

In [ ]:
def esp(seed, ndcat):
    ndcat.random = np.random.RandomState(seed)
    x = [ndcat.new() for _ in range(1000)]

    def fun(delta, k):
        ndcat.delta = [delta]*len(ndcat.solutions)
        ndcat.seed = seed
        for y in x:
            y[0] = k
        return np.mean([ndcat.distance(x, ndcat.mutate(x)) for x in x])
    return fun

#### optimizer

In [ ]:
ndcat = ndCategory({'Adam', 'Nadam', 'RMSprop', 'SGD'},
                   Adam=floatBinary(0, 0.1, digits=4),
                   Nadam=floatBinary(0, 0.1, digits=4),
                   RMSprop=ndSolution(floatBinary(0, 0.1, digits=4),
                                      floatBinary(0, 1, digits=3)),
                   SGD=ndSolution(floatBinary(0, 0.1, digits=4),
                                  floatBinary(0, 1, digits=3)))

sol = floatBinary(1, 2, digits=4).agg('neighbor')
sample = 50
history = []
for i in range(sample):
    seed = 1679 + i
    feasible_factor = esp(seed, ndcat)
    for k in ndcat.solutions.keys():
        session = LocalSearch(lambda delta: abs(1-feasible_factor(delta, k)), sol, itermax=250,
                              seed=seed, cooling=Cooling.Exp(0.001, itermax=200)).session
        history.append({'name': k, 'fmin': session.fmin,
                        'xmin': sol.decode(sol.read(session.xmin)), 'seed': seed})
history = pd.DataFrame(history)
history[history.fmin == 0].groupby('name').xmin.describe()

#### activation

In [ ]:
ndcat = ndCategory({'ELU', 'LeakyReLU'},
                   ELU=floatBinary(0, 0.5, digits=4),
                   LeakyReLU=floatBinary(0, 0.5, digits=4))

sol = floatBinary(1, 2, digits=4).agg('neighbor')
sample = 50
history = []
for i in range(sample):
    seed = 1679 + i
    feasible_factor = esp(seed, ndcat)
    for k in ndcat.solutions.keys():
        session = LocalSearch(lambda delta: abs(1-feasible_factor(delta, k)), sol, itermax=250,
                              seed=seed, cooling=Cooling.Exp(0.001, itermax=200)).session
        history.append({'name': k, 'fmin': session.fmin,
                        'xmin': sol.decode(sol.read(session.xmin)), 'seed': seed})
history = pd.DataFrame(history)
history[history.fmin == 0].groupby('name').xmin.describe()

#### regularization

In [ ]:
ndcat = ndCategory({'l1', 'l2', 'None'},
                   l1=floatBinary(0, 0.1, digits=4),
                   l2=floatBinary(0, 0.1, digits=4))

sol = floatBinary(1, 2, digits=4).agg('neighbor')
sample = 50
history = []
for i in range(sample):
    seed = 1679 + i
    feasible_factor = esp(seed, ndcat)
    for k in ndcat.solutions.keys():
        session = LocalSearch(lambda delta: abs(1-feasible_factor(delta, k)), sol, itermax=250,
                              seed=seed, cooling=Cooling.Exp(0.001, itermax=200)).session
        history.append({'name': k, 'fmin': session.fmin,
                        'xmin': sol.decode(sol.read(session.xmin)), 'seed': seed})
history = pd.DataFrame(history)
history[history.fmin == 0].groupby('name').xmin.describe()

#### dropout

In [ ]:
ndcat = ndCategory({'true': True, 'false': False},
                   true=floatBinary(0, 0.5, digits=4))

sol = floatBinary(1, 2, digits=4).agg('neighbor')
sample = 50
history = []
for i in range(sample):
    seed = 1679 + i
    feasible_factor = esp(seed, ndcat)
    for k in ndcat.solutions.keys():
        session = LocalSearch(lambda delta: abs(1-feasible_factor(delta, k)), sol, itermax=250,
                              seed=seed, cooling=Cooling.Exp(0.001, itermax=200)).session
        history.append({'name': k, 'fmin': session.fmin,
                        'xmin': sol.decode(sol.read(session.xmin)), 'seed': seed})
history = pd.DataFrame(history)
history[history.fmin == 0].groupby('name').xmin.describe()

--------------------------------------

## Cálculo de probabilidad de soluciones factibles

In [3]:
from src.python.optimize.search import SearchSpace


class SearchSpace_(SearchSpace):

    def __init__(self, seed=None, scale=None) -> None:
        solution = ndSolution(
            neurons=intBinary(1, 2**9, digits=0),
            deep=intBinary(1, 2**4, digits=0),  # x <= 9
            optimizer=ndCategory({'Adam', 'Nadam', 'RMSprop', 'SGD'},
                                 Adam=floatBinary(0, 0.1, digits=4),  # x > 0
                                 Nadam=floatBinary(0, 0.1, digits=4),  # x > 0
                                 RMSprop=ndSolution(floatBinary(0, 0.1, digits=4),  # x > 0
                                                    floatBinary(0, 1, digits=3)),
                                 SGD=ndSolution(floatBinary(0, 0.1, digits=4),  # x > 0
                                                floatBinary(0, 1, digits=3)),
                                 solname='optimizer_param',
                                 delta=(1.316670, 1.316670, 1.185833, 1.185833)),
            activation=ndCategory({'ELU', 'LeakyReLU'},
                                  ELU=floatBinary(0, 0.5, digits=4),
                                  LeakyReLU=floatBinary(0, 0.5, digits=4),
                                  solname='activation_alpha',
                                  delta=1.091558),
            initializer=Category({'uniform', 'normal'}),
            batch_size=intBinary(1, 2**8, digits=0),  # x >= 8
            batch_normalization=Category({True, False}),
            regularization=ndCategory({'l1', 'l2', 'None'},
                                      l1=floatBinary(0, 0.1, digits=4),
                                      l2=floatBinary(0, 0.1, digits=4),
                                      solname='regularization_factor',
                                      delta=1.195569),
            dropout=ndCategory({'true': True, 'false': False},
                               true=floatBinary(0, 0.5, digits=4),
                               solname='dropout_rate',
                               delta=1.091558)
        )
        super().__init__(solution, seed)

    def is_feasible(self, x):
        x = self.solution.decode(x, True)
        maxdeep = 1 + math.log(x['neurons'] / 2.0) / math.log(2)
        return \
            x['deep'] <= maxdeep and\
            x['optimizer_param'][0] > 0 and\
            x['optimizer_param'][1] > 0 and\
            x['optimizer_param'][2][0] > 0 and\
            x['optimizer_param'][3][0] > 0 and\
            x['batch_size'] >= 8


search_space = SearchSpace_()
solution = search_space.solution
solution.agg('mutate', scale=1 / len(solution))

(neurons=intBinary(1, 512, 9), deep=intBinary(1, 16, 4), optimizer=Category({Adam, Nadam, RMSprop, SGD}, Adam=floatBinary(0.0, 0.1, 10), Nadam=floatBinary(0.0, 0.1, 10), RMSprop=(floatBinary(0.0, 0.1, 10), floatBinary(0.0, 1.0, 10)), SGD=(floatBinary(0.0, 0.1, 10), floatBinary(0.0, 1.0, 10))), activation=Category({ELU, LeakyReLU}, ELU=floatBinary(0.0, 0.5, 13), LeakyReLU=floatBinary(0.0, 0.5, 13)), initializer=Category({normal, uniform}), batch_size=intBinary(1, 256, 8), batch_normalization=Category({False, True}), regularization=Category({None, l1, l2}, l1=floatBinary(0.0, 0.1, 10), l2=floatBinary(0.0, 0.1, 10)), dropout=Category({false, true}, true=floatBinary(0.0, 0.5, 13)))